# Projet SQL — Thème B : Gestion d’un festival de musique (SQLite)



In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

# =========================
# Fonction utilitaire : exécuter une requête + afficher résultat
# =========================
def run_query(conn, sql, params=None):
    """
    Exécute une requête SQL et renvoie un DataFrame.
    - sql : requête SQL (SELECT)
    - params : paramètres optionnels pour requêtes paramétrées
    """
    if params is None:
        params = ()
    return pd.read_sql_query(sql, conn, params=params)

# =========================
# Connexion SQLite
# =========================
conn = sqlite3.connect(":memory:")   # ou "festival.db" si tu veux enregistrer sur disque
conn.execute("PRAGMA foreign_keys = ON;")
print("Connexion SQLite OK (PRAGMA foreign_keys=ON)")


✅ Connexion SQLite OK (PRAGMA foreign_keys=ON)


## 1) Création des tables + contraintes (CMD)

In [6]:
schema_sql = """
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS Billet;
DROP TABLE IF EXISTS Concert;
DROP TABLE IF EXISTS Participant;
DROP TABLE IF EXISTS Artiste;
DROP TABLE IF EXISTS Scene;
DROP TABLE IF EXISTS Organisateur;

CREATE TABLE Organisateur (
  id_organisateur  INTEGER PRIMARY KEY AUTOINCREMENT,
  nom_organisation TEXT NOT NULL,
  responsable      TEXT,
  email_contact    TEXT NOT NULL UNIQUE,
  telephone        TEXT,
  specialite       TEXT
);

CREATE TABLE Artiste (
  id_artiste       INTEGER PRIMARY KEY AUTOINCREMENT,
  ref_organisateur INTEGER NOT NULL,
  nom_artiste      TEXT NOT NULL,
  genre_musical    TEXT,
  pays_origine     TEXT,
  nombre_membres   INTEGER CHECK (nombre_membres >= 1),
  contact_email    TEXT UNIQUE,
  FOREIGN KEY (ref_organisateur) REFERENCES Organisateur(id_organisateur)
    ON UPDATE CASCADE ON DELETE RESTRICT
);

CREATE TABLE Scene (
  id_scene         INTEGER PRIMARY KEY AUTOINCREMENT,
  ref_organisateur INTEGER NOT NULL,
  nom_scene        TEXT NOT NULL,
  capacite_max     INTEGER NOT NULL CHECK (capacite_max > 0),
  type_scene       TEXT,
  couvert          INTEGER CHECK (couvert IN (0,1)),
  equipement       TEXT,
  UNIQUE (ref_organisateur, nom_scene),
  FOREIGN KEY (ref_organisateur) REFERENCES Organisateur(id_organisateur)
    ON UPDATE CASCADE ON DELETE RESTRICT
);

CREATE TABLE Concert (
  id_concert       INTEGER PRIMARY KEY AUTOINCREMENT,
  ref_artiste      INTEGER NOT NULL,
  ref_scene        INTEGER NOT NULL,
  date_heure_debut TEXT NOT NULL,
  date_heure_fin   TEXT,
  duree_minutes    INTEGER CHECK (duree_minutes >= 0),
  statut           TEXT,
  FOREIGN KEY (ref_artiste) REFERENCES Artiste(id_artiste)
    ON UPDATE CASCADE ON DELETE RESTRICT,
  FOREIGN KEY (ref_scene) REFERENCES Scene(id_scene)
    ON UPDATE CASCADE ON DELETE RESTRICT
);

CREATE TABLE Participant (
  id_participant   INTEGER PRIMARY KEY AUTOINCREMENT,
  nom              TEXT NOT NULL,
  prenom           TEXT,
  email            TEXT UNIQUE,
  telephone        TEXT,
  date_naissance   TEXT,
  ville            TEXT
);

CREATE TABLE Billet (
  id_billet        INTEGER PRIMARY KEY AUTOINCREMENT,
  ref_concert      INTEGER NOT NULL,
  ref_participant  INTEGER NOT NULL,
  type_billet      TEXT NOT NULL,
  prix             REAL NOT NULL CHECK (prix >= 0),
  date_achat       TEXT,
  statut_paiement  TEXT,
  FOREIGN KEY (ref_concert) REFERENCES Concert(id_concert)
    ON UPDATE CASCADE ON DELETE CASCADE,
  FOREIGN KEY (ref_participant) REFERENCES Participant(id_participant)
    ON UPDATE CASCADE ON DELETE CASCADE
);
"""

conn.executescript(schema_sql)
conn.commit()
print(" Tables + contraintes créées (conformes MCD/MLD V2)")


 Tables + contraintes créées (conformes MCD/MLD V2)


## 2) Insertion des données (LMD)
Objectif : **180 lignes insérées au total** avec **beaucoup de participants et de billets**, et **peu de scènes**.

In [7]:
inserts_sql = r"""
-- =========================
-- ORGANISATEUR (3)
-- =========================
INSERT INTO Organisateur(nom_organisation, responsable, email_contact, telephone, specialite) VALUES
  ('Org_NordSound','Resp Nord','contact@nordsound.com','0700000001','Rock'),
  ('Org_ElectroWave','Resp Electro','info@electrowave.com','0700000002','Electro'),
  ('Org_IndiePulse','Resp Indie','contact@indiepulse.com','0700000003','Indie');

-- =========================
-- SCENE (3)  (réduit)
-- ref_organisateur : 1..3
-- =========================
INSERT INTO Scene(ref_organisateur, nom_scene, capacite_max, type_scene, couvert, equipement) VALUES
  (1,'Grande Scene',15000,'Main',0,'Son + Lumiere'),
  (2,'Electro Dome',9000,'Dome',1,'LED + Laser'),
  (3,'Indie Garden',6000,'Outdoor',0,'Acoustique');

-- =========================
-- ARTISTE (12)
-- ref_organisateur : réparti sur 1..3
-- =========================
INSERT INTO Artiste(ref_organisateur, nom_artiste, genre_musical, pays_origine, nombre_membres, contact_email) VALUES
  (1,'Artiste_01','Rock','France',4,'a01@mail.com'),
  (2,'Artiste_02','Electro','Belgique',1,'a02@mail.com'),
  (3,'Artiste_03','Pop','Royaume-Uni',5,'a03@mail.com'),
  (1,'Artiste_04','Jazz','France',3,'a04@mail.com'),
  (2,'Artiste_05','Hip-hop','USA',2,'a05@mail.com'),
  (3,'Artiste_06','Folk','Canada',4,'a06@mail.com'),
  (1,'Artiste_07','Metal','Allemagne',5,'a07@mail.com'),
  (2,'Artiste_08','R&B','Belgique',1,'a08@mail.com'),
  (3,'Artiste_09','Rock','Italie',4,'a09@mail.com'),
  (1,'Artiste_10','Electro','France',1,'a10@mail.com'),
  (2,'Artiste_11','Pop','Espagne',3,'a11@mail.com'),
  (3,'Artiste_12','Jazz','France',6,'a12@mail.com');

-- =========================
-- CONCERT (12)
-- (artistes 1..4 ont 3 concerts chacun -> utile pour HAVING >= 3)
-- scènes cyclées 1..3
-- Schéma V2 : ref_artiste, ref_scene, date_heure_debut, date_heure_fin, duree_minutes, statut
-- =========================
INSERT INTO Concert(ref_artiste, ref_scene, date_heure_debut, date_heure_fin, duree_minutes, statut) VALUES
  (1, 1, '2025-07-18 16:00:00', '2025-07-18 17:30:00', 90, 'confirme'),
  (1, 2, '2025-07-18 18:00:00', '2025-07-18 19:30:00', 90, 'confirme'),
  (1, 3, '2025-07-18 20:00:00', '2025-07-18 21:30:00', 90, 'confirme'),

  (2, 1, '2025-07-19 16:00:00', '2025-07-19 17:30:00', 90, 'confirme'),
  (2, 2, '2025-07-19 18:00:00', '2025-07-19 19:30:00', 90, 'confirme'),
  (2, 3, '2025-07-19 20:00:00', '2025-07-19 21:30:00', 90, 'confirme'),

  (3, 1, '2025-07-20 16:00:00', '2025-07-20 17:30:00', 90, 'confirme'),
  (3, 2, '2025-07-20 18:00:00', '2025-07-20 19:30:00', 90, 'confirme'),
  (3, 3, '2025-07-20 20:00:00', '2025-07-20 21:30:00', 90, 'confirme'),

  (4, 1, '2025-07-21 16:00:00', '2025-07-21 17:30:00', 90, 'confirme'),
  (4, 2, '2025-07-21 18:00:00', '2025-07-21 19:30:00', 90, 'confirme'),
  (4, 3, '2025-07-21 20:00:00', '2025-07-21 21:30:00', 90, 'confirme');

-- =========================
-- PARTICIPANT (90)
-- Schéma V2 : nom, prenom, email, telephone, date_naissance, ville
-- =========================
INSERT INTO Participant(nom, prenom, email, telephone, date_naissance, ville) VALUES
  ('Participant_001','Prenom001','p001@mail.com','0600000001','2001-01-01','Lille'),
  ('Participant_002','Prenom002','p002@mail.com','0600000002','2001-01-02','Lille'),
  ('Participant_003','Prenom003','p003@mail.com','0600000003','2001-01-03','Roubaix'),
  ('Participant_004','Prenom004','p004@mail.com','0600000004','2001-01-04','Roubaix'),
  ('Participant_005','Prenom005','p005@mail.com','0600000005','2001-01-05','Tourcoing'),
  ('Participant_006','Prenom006','p006@mail.com','0600000006','2001-01-06','Tourcoing'),
  ('Participant_007','Prenom007','p007@mail.com','0600000007','2001-01-07','Villeneuve-d''Ascq'),
  ('Participant_008','Prenom008','p008@mail.com','0600000008','2001-01-08','Villeneuve-d''Ascq'),
  ('Participant_009','Prenom009','p009@mail.com','0600000009','2001-01-09','Lille'),
  ('Participant_010','Prenom010','p010@mail.com','0600000010','2001-01-10','Lille'),

  ('Participant_011','Prenom011','p011@mail.com','0600000011','2001-01-11','Roubaix'),
  ('Participant_012','Prenom012','p012@mail.com','0600000012','2001-01-12','Roubaix'),
  ('Participant_013','Prenom013','p013@mail.com','0600000013','2001-01-13','Tourcoing'),
  ('Participant_014','Prenom014','p014@mail.com','0600000014','2001-01-14','Tourcoing'),
  ('Participant_015','Prenom015','p015@mail.com','0600000015','2001-01-15','Villeneuve-d''Ascq'),
  ('Participant_016','Prenom016','p016@mail.com','0600000016','2001-01-16','Villeneuve-d''Ascq'),
  ('Participant_017','Prenom017','p017@mail.com','0600000017','2001-01-17','Lille'),
  ('Participant_018','Prenom018','p018@mail.com','0600000018','2001-01-18','Lille'),
  ('Participant_019','Prenom019','p019@mail.com','0600000019','2001-01-19','Roubaix'),
  ('Participant_020','Prenom020','p020@mail.com','0600000020','2001-01-20','Roubaix'),

  ('Participant_021','Prenom021','p021@mail.com','0600000021','2001-01-21','Tourcoing'),
  ('Participant_022','Prenom022','p022@mail.com','0600000022','2001-01-22','Tourcoing'),
  ('Participant_023','Prenom023','p023@mail.com','0600000023','2001-01-23','Villeneuve-d''Ascq'),
  ('Participant_024','Prenom024','p024@mail.com','0600000024','2001-01-24','Villeneuve-d''Ascq'),
  ('Participant_025','Prenom025','p025@mail.com','0600000025','2001-01-25','Lille'),
  ('Participant_026','Prenom026','p026@mail.com','0600000026','2001-01-26','Lille'),
  ('Participant_027','Prenom027','p027@mail.com','0600000027','2001-01-27','Roubaix'),
  ('Participant_028','Prenom028','p028@mail.com','0600000028','2001-01-28','Roubaix'),
  ('Participant_029','Prenom029','p029@mail.com','0600000029','2001-01-29','Tourcoing'),
  ('Participant_030','Prenom030','p030@mail.com','0600000030','2001-01-30','Tourcoing'),

  ('Participant_031','Prenom031','p031@mail.com','0600000031','2001-02-01','Villeneuve-d''Ascq'),
  ('Participant_032','Prenom032','p032@mail.com','0600000032','2001-02-02','Villeneuve-d''Ascq'),
  ('Participant_033','Prenom033','p033@mail.com','0600000033','2001-02-03','Lille'),
  ('Participant_034','Prenom034','p034@mail.com','0600000034','2001-02-04','Lille'),
  ('Participant_035','Prenom035','p035@mail.com','0600000035','2001-02-05','Roubaix'),
  ('Participant_036','Prenom036','p036@mail.com','0600000036','2001-02-06','Roubaix'),
  ('Participant_037','Prenom037','p037@mail.com','0600000037','2001-02-07','Tourcoing'),
  ('Participant_038','Prenom038','p038@mail.com','0600000038','2001-02-08','Tourcoing'),
  ('Participant_039','Prenom039','p039@mail.com','0600000039','2001-02-09','Villeneuve-d''Ascq'),
  ('Participant_040','Prenom040','p040@mail.com','0600000040','2001-02-10','Villeneuve-d''Ascq'),

  ('Participant_041','Prenom041','p041@mail.com','0600000041','2001-02-11','Lille'),
  ('Participant_042','Prenom042','p042@mail.com','0600000042','2001-02-12','Lille'),
  ('Participant_043','Prenom043','p043@mail.com','0600000043','2001-02-13','Roubaix'),
  ('Participant_044','Prenom044','p044@mail.com','0600000044','2001-02-14','Roubaix'),
  ('Participant_045','Prenom045','p045@mail.com','0600000045','2001-02-15','Tourcoing'),
  ('Participant_046','Prenom046','p046@mail.com','0600000046','2001-02-16','Tourcoing'),
  ('Participant_047','Prenom047','p047@mail.com','0600000047','2001-02-17','Villeneuve-d''Ascq'),
  ('Participant_048','Prenom048','p048@mail.com','0600000048','2001-02-18','Villeneuve-d''Ascq'),
  ('Participant_049','Prenom049','p049@mail.com','0600000049','2001-02-19','Lille'),
  ('Participant_050','Prenom050','p050@mail.com','0600000050','2001-02-20','Lille'),

  ('Participant_051','Prenom051','p051@mail.com','0600000051','2001-02-21','Roubaix'),
  ('Participant_052','Prenom052','p052@mail.com','0600000052','2001-02-22','Roubaix'),
  ('Participant_053','Prenom053','p053@mail.com','0600000053','2001-02-23','Tourcoing'),
  ('Participant_054','Prenom054','p054@mail.com','0600000054','2001-02-24','Tourcoing'),
  ('Participant_055','Prenom055','p055@mail.com','0600000055','2001-02-25','Villeneuve-d''Ascq'),
  ('Participant_056','Prenom056','p056@mail.com','0600000056','2001-02-26','Villeneuve-d''Ascq'),
  ('Participant_057','Prenom057','p057@mail.com','0600000057','2001-02-27','Lille'),
  ('Participant_058','Prenom058','p058@mail.com','0600000058','2001-02-28','Lille'),
  ('Participant_059','Prenom059','p059@mail.com','0600000059','2001-03-01','Roubaix'),
  ('Participant_060','Prenom060','p060@mail.com','0600000060','2001-03-02','Roubaix'),

  ('Participant_061','Prenom061','p061@mail.com','0600000061','2001-03-03','Tourcoing'),
  ('Participant_062','Prenom062','p062@mail.com','0600000062','2001-03-04','Tourcoing'),
  ('Participant_063','Prenom063','p063@mail.com','0600000063','2001-03-05','Villeneuve-d''Ascq'),
  ('Participant_064','Prenom064','p064@mail.com','0600000064','2001-03-06','Villeneuve-d''Ascq'),
  ('Participant_065','Prenom065','p065@mail.com','0600000065','2001-03-07','Lille'),
  ('Participant_066','Prenom066','p066@mail.com','0600000066','2001-03-08','Lille'),
  ('Participant_067','Prenom067','p067@mail.com','0600000067','2001-03-09','Roubaix'),
  ('Participant_068','Prenom068','p068@mail.com','0600000068','2001-03-10','Roubaix'),
  ('Participant_069','Prenom069','p069@mail.com','0600000069','2001-03-11','Tourcoing'),
  ('Participant_070','Prenom070','p070@mail.com','0600000070','2001-03-12','Tourcoing'),

  ('Participant_071','Prenom071','p071@mail.com','0600000071','2001-03-13','Villeneuve-d''Ascq'),
  ('Participant_072','Prenom072','p072@mail.com','0600000072','2001-03-14','Villeneuve-d''Ascq'),
  ('Participant_073','Prenom073','p073@mail.com','0600000073','2001-03-15','Lille'),
  ('Participant_074','Prenom074','p074@mail.com','0600000074','2001-03-16','Lille'),
  ('Participant_075','Prenom075','p075@mail.com','0600000075','2001-03-17','Roubaix'),
  ('Participant_076','Prenom076','p076@mail.com','0600000076','2001-03-18','Roubaix'),
  ('Participant_077','Prenom077','p077@mail.com','0600000077','2001-03-19','Tourcoing'),
  ('Participant_078','Prenom078','p078@mail.com','0600000078','2001-03-20','Tourcoing'),
  ('Participant_079','Prenom079','p079@mail.com','0600000079','2001-03-21','Villeneuve-d''Ascq'),
  ('Participant_080','Prenom080','p080@mail.com','0600000080','2001-03-22','Villeneuve-d''Ascq'),

  ('Participant_081','Prenom081','p081@mail.com','0600000081','2001-03-23','Lille'),
  ('Participant_082','Prenom082','p082@mail.com','0600000082','2001-03-24','Lille'),
  ('Participant_083','Prenom083','p083@mail.com','0600000083','2001-03-25','Roubaix'),
  ('Participant_084','Prenom084','p084@mail.com','0600000084','2001-03-26','Roubaix'),
  ('Participant_085','Prenom085','p085@mail.com','0600000085','2001-03-27','Tourcoing'),
  ('Participant_086','Prenom086','p086@mail.com','0600000086','2001-03-28','Tourcoing'),
  ('Participant_087','Prenom087','p087@mail.com','0600000087','2001-03-29','Villeneuve-d''Ascq'),
  ('Participant_088','Prenom088','p088@mail.com','0600000088','2001-03-30','Villeneuve-d''Ascq'),
  ('Participant_089','Prenom089','p089@mail.com','0600000089','2001-03-31','Lille'),
  ('Participant_090','Prenom090','p090@mail.com','0600000090','2001-04-01','Lille');

-- =========================
-- BILLET (60)
-- Schéma V2 : ref_concert, ref_participant, type_billet, prix, date_achat, statut_paiement
-- On varie type_billet + paiement pour rendre la base plus réaliste
-- =========================
INSERT INTO Billet(ref_concert, ref_participant, type_billet, prix, date_achat, statut_paiement) VALUES
  (1,  1, 'Standard', 65.0, '2025-06-01', 'paye'),
  (1,  2, 'VIP',      90.0, '2025-06-02', 'paye'),
  (1,  3, 'Reduit',   55.0, '2025-06-03', 'paye'),

  (2,  4, 'Standard', 55.0, '2025-06-01', 'paye'),
  (2,  5, 'VIP',      80.0, '2025-06-02', 'paye'),
  (2,  6, 'Reduit',   45.0, '2025-06-03', 'paye'),

  (3,  7, 'Standard', 45.0, '2025-06-01', 'paye'),
  (3,  8, 'VIP',      70.0, '2025-06-02', 'paye'),
  (3,  9, 'Reduit',   35.0, '2025-06-03', 'paye'),

  (4, 10, 'Standard', 65.0, '2025-06-04', 'paye'),
  (4, 11, 'VIP',      90.0, '2025-06-05', 'paye'),
  (4, 12, 'Reduit',   55.0, '2025-06-06', 'paye'),

  (5, 13, 'Standard', 55.0, '2025-06-04', 'paye'),
  (5, 14, 'VIP',      80.0, '2025-06-05', 'paye'),
  (5, 15, 'Reduit',   45.0, '2025-06-06', 'paye'),

  (6, 16, 'Standard', 45.0, '2025-06-04', 'paye'),
  (6, 17, 'VIP',      70.0, '2025-06-05', 'paye'),
  (6, 18, 'Reduit',   35.0, '2025-06-06', 'paye'),

  (7, 19, 'Standard', 65.0, '2025-06-07', 'paye'),
  (7, 20, 'VIP',      90.0, '2025-06-08', 'paye'),
  (7, 21, 'Reduit',   55.0, '2025-06-09', 'paye'),

  (8, 22, 'Standard', 55.0, '2025-06-07', 'paye'),
  (8, 23, 'VIP',      80.0, '2025-06-08', 'paye'),
  (8, 24, 'Reduit',   45.0, '2025-06-09', 'paye'),

  (9, 25, 'Standard', 45.0, '2025-06-07', 'paye'),
  (9, 26, 'VIP',      70.0, '2025-06-08', 'paye'),
  (9, 27, 'Reduit',   35.0, '2025-06-09', 'paye'),

  (10, 28, 'Standard', 65.0, '2025-06-10', 'paye'),
  (10, 29, 'VIP',      90.0, '2025-06-11', 'paye'),
  (10, 30, 'Reduit',   55.0, '2025-06-12', 'paye'),

  (11, 31, 'Standard', 55.0, '2025-06-10', 'paye'),
  (11, 32, 'VIP',      80.0, '2025-06-11', 'paye'),
  (11, 33, 'Reduit',   45.0, '2025-06-12', 'paye'),

  (12, 34, 'Standard', 45.0, '2025-06-10', 'paye'),
  (12, 35, 'VIP',      70.0, '2025-06-11', 'paye'),
  (12, 36, 'Reduit',   35.0, '2025-06-12', 'paye'),

  (1, 37, 'Standard', 65.0, '2025-06-13', 'paye'),
  (1, 38, 'VIP',      90.0, '2025-06-13', 'paye'),

  (2, 39, 'Standard', 55.0, '2025-06-14', 'paye'),
  (2, 40, 'VIP',      80.0, '2025-06-14', 'paye'),

  (3, 41, 'Standard', 45.0, '2025-06-15', 'paye'),
  (3, 42, 'VIP',      70.0, '2025-06-15', 'paye'),

  (4, 43, 'Standard', 65.0, '2025-06-16', 'paye'),
  (4, 44, 'VIP',      90.0, '2025-06-16', 'paye'),

  (5, 45, 'Standard', 55.0, '2025-06-17', 'paye'),
  (5, 46, 'VIP',      80.0, '2025-06-17', 'paye'),

  (6, 47, 'Standard', 45.0, '2025-06-18', 'paye'),
  (6, 48, 'VIP',      70.0, '2025-06-18', 'paye'),

  (7, 49, 'Standard', 65.0, '2025-06-19', 'paye'),
  (7, 50, 'VIP',      90.0, '2025-06-19', 'paye'),

  (8, 51, 'Standard', 55.0, '2025-06-20', 'paye'),
  (8, 52, 'VIP',      80.0, '2025-06-20', 'paye'),

  (9, 53, 'Standard', 45.0, '2025-06-21', 'paye'),
  (9, 54, 'VIP',      70.0, '2025-06-21', 'paye'),

  (10, 55, 'Standard', 65.0, '2025-06-22', 'paye'),
  (10, 56, 'VIP',      90.0, '2025-06-22', 'paye'),

  (11, 57, 'Standard', 55.0, '2025-06-23', 'paye'),
  (11, 58, 'VIP',      80.0, '2025-06-23', 'paye'),

  (12, 59, 'Standard', 45.0, '2025-06-24', 'paye'),
  (12, 60, 'VIP',      70.0, '2025-06-24', 'paye');
"""

conn.executescript(inserts_sql)
conn.commit()
print(" Données insérées (base cohérente avec le schéma V2)")


 Données insérées (base cohérente avec le schéma V2)


### Vérification : volumes (doit être entre 50 et 200 lignes au total)

In [8]:
# =========================
# Vérification : nombre de lignes par table
# =========================
tables = ["Organisateur","Scene","Artiste","Concert","Participant","Billet"]

for t in tables:
    nb = conn.execute(f"SELECT COUNT(*) FROM {t}").fetchone()[0]
    print(f"{t:<12} : {nb}")




Organisateur : 3
Scene        : 3
Artiste      : 12
Concert      : 12
Participant  : 90
Billet       : 60


## 3) Requêtes obligatoires

In [9]:
# Q1 — Sélection simple : toutes les scènes (capacité décroissante)
run_query(conn, """
SELECT id_scene, nom_scene, capacite_max
FROM Scene
ORDER BY capacite_max DESC;
""")

,id_scene,nom_scene,capacite_max
0,1,Grande Scene,15000
1,2,Electro Dome,9000
2,3,Indie Garden,6000


In [10]:
# Q2 — Filtrage multi-conditions : artistes (genre + pays)
run_query(conn, """
SELECT id_artiste, nom_artiste, genre_musical, pays_origine
FROM Artiste
WHERE genre_musical IN ('Electro','Rock')
  AND pays_origine IN ('France','Belgique')
ORDER BY nom_artiste;
""")



,id_artiste,nom_artiste,genre_musical,pays_origine
0,1,Artiste_01,Rock,France
1,2,Artiste_02,Electro,Belgique
2,10,Artiste_10,Electro,France


In [11]:
# Q3 — JOIN (INNER) : programmation (concerts) avec scène + artiste
run_query(conn, """
SELECT
    c.id_concert,
    s.nom_scene      AS scene,
    a.nom_artiste    AS artiste,
    a.genre_musical  AS genre,
    c.date_heure_debut AS debut
FROM Concert c
JOIN Scene s   ON s.id_scene   = c.ref_scene
JOIN Artiste a ON a.id_artiste = c.ref_artiste
ORDER BY c.date_heure_debut, s.nom_scene
LIMIT 25;
""")



,id_concert,scene,artiste,genre,debut
0,1,Grande Scene,Artiste_01,Rock,2025-07-18 16:00:00
1,2,Electro Dome,Artiste_01,Rock,2025-07-18 18:00:00
2,3,Indie Garden,Artiste_01,Rock,2025-07-18 20:00:00
3,4,Grande Scene,Artiste_02,Electro,2025-07-19 16:00:00
4,5,Electro Dome,Artiste_02,Electro,2025-07-19 18:00:00
5,6,Indie Garden,Artiste_02,Electro,2025-07-19 20:00:00
6,7,Grande Scene,Artiste_03,Pop,2025-07-20 16:00:00
7,8,Electro Dome,Artiste_03,Pop,2025-07-20 18:00:00
8,9,Indie Garden,Artiste_03,Pop,2025-07-20 20:00:00
9,10,Grande Scene,Artiste_04,Jazz,2025-07-21 16:00:00


In [12]:
# Q4 — Agrégats : nombre de concerts par scène
run_query(conn, """
SELECT
    s.nom_scene AS scene,
    COUNT(c.id_concert) AS nb_concerts
FROM Concert c
JOIN Scene s ON s.id_scene = c.ref_scene
GROUP BY s.id_scene, s.nom_scene
ORDER BY nb_concerts DESC, scene;
""")



,scene,nb_concerts
0,Electro Dome,4
1,Grande Scene,4
2,Indie Garden,4


In [13]:
# Q5 — GROUP BY + HAVING : artistes ayant au moins 3 concerts
run_query(conn, """
SELECT
    a.nom_artiste AS artiste,
    COUNT(c.id_concert) AS nb_concerts
FROM Concert c
JOIN Artiste a ON a.id_artiste = c.ref_artiste
GROUP BY a.id_artiste, a.nom_artiste
HAVING COUNT(c.id_concert) >= 3
ORDER BY nb_concerts DESC, artiste;
""")


,artiste,nb_concerts
0,Artiste_01,3
1,Artiste_02,3
2,Artiste_03,3
3,Artiste_04,3


In [14]:
# Q6 — Sous-requête : concerts sur la plus grande scène (capacité max)
run_query(conn, """
SELECT
    c.id_concert,
    s.nom_scene AS scene,
    a.nom_artiste AS artiste,
    c.date_heure_debut AS debut
FROM Concert c
JOIN Scene s   ON s.id_scene = c.ref_scene
JOIN Artiste a ON a.id_artiste = c.ref_artiste
WHERE s.capacite_max = (SELECT MAX(capacite_max) FROM Scene)
ORDER BY c.date_heure_debut;
""")


,id_concert,scene,artiste,debut
0,1,Grande Scene,Artiste_01,2025-07-18 16:00:00
1,4,Grande Scene,Artiste_02,2025-07-19 16:00:00
2,7,Grande Scene,Artiste_03,2025-07-20 16:00:00
3,10,Grande Scene,Artiste_04,2025-07-21 16:00:00


In [15]:
# Q7 — JOIN (LEFT) : participants et nb de billets (inclut ceux avec 0 billet)
run_query(conn, """
SELECT
    p.id_participant,
    p.nom,
    COUNT(b.id_billet) AS nb_billets,
    COALESCE(SUM(b.prix), 0) AS total_depense
FROM Participant p
LEFT JOIN Billet b ON b.ref_participant = p.id_participant
GROUP BY p.id_participant, p.nom
ORDER BY nb_billets DESC, total_depense DESC, p.nom
LIMIT 25;
""")


,id_participant,nom,nb_billets,total_depense
0,2,Participant_002,1,90.0
1,11,Participant_011,1,90.0
2,20,Participant_020,1,90.0
3,29,Participant_029,1,90.0
4,38,Participant_038,1,90.0
5,44,Participant_044,1,90.0
6,50,Participant_050,1,90.0
7,56,Participant_056,1,90.0
8,5,Participant_005,1,80.0
9,14,Participant_014,1,80.0


In [16]:
# Q8 — VUE : programmation lisible
conn.execute("DROP VIEW IF EXISTS v_programmation;")
conn.execute("""
CREATE VIEW v_programmation AS
SELECT
    c.id_concert,
    date(c.date_heure_debut) AS jour,
    time(c.date_heure_debut) AS heure,
    s.nom_scene              AS scene,
    a.nom_artiste            AS artiste,
    a.genre_musical          AS genre
FROM Concert c
JOIN Scene s   ON s.id_scene   = c.ref_scene
JOIN Artiste a ON a.id_artiste = c.ref_artiste;
""")

run_query(conn, """
SELECT *
FROM v_programmation
ORDER BY jour, heure, scene
LIMIT 20;
""")


,id_concert,jour,heure,scene,artiste,genre
0,1,2025-07-18,16:00:00,Grande Scene,Artiste_01,Rock
1,2,2025-07-18,18:00:00,Electro Dome,Artiste_01,Rock
2,3,2025-07-18,20:00:00,Indie Garden,Artiste_01,Rock
3,4,2025-07-19,16:00:00,Grande Scene,Artiste_02,Electro
4,5,2025-07-19,18:00:00,Electro Dome,Artiste_02,Electro
5,6,2025-07-19,20:00:00,Indie Garden,Artiste_02,Electro
6,7,2025-07-20,16:00:00,Grande Scene,Artiste_03,Pop
7,8,2025-07-20,18:00:00,Electro Dome,Artiste_03,Pop
8,9,2025-07-20,20:00:00,Indie Garden,Artiste_03,Pop
9,10,2025-07-21,16:00:00,Grande Scene,Artiste_04,Jazz


In [17]:
# Requête avancée : CTE + fenêtrage (RANK) — classement des artistes par nb de concerts
run_query(conn, """
WITH stats AS (
  SELECT
      a.id_artiste,
      a.nom_artiste AS artiste,
      a.genre_musical AS genre,
      COUNT(c.id_concert) AS nb_concerts
  FROM Concert c
  JOIN Artiste a ON a.id_artiste = c.ref_artiste
  GROUP BY a.id_artiste, a.nom_artiste, a.genre_musical
),
classement AS (
  SELECT
      *,
      RANK() OVER (ORDER BY nb_concerts DESC) AS rang
  FROM stats
)
SELECT artiste, genre, nb_concerts, rang
FROM classement
ORDER BY rang, artiste;
""")


,artiste,genre,nb_concerts,rang
0,Artiste_01,Rock,3,1
1,Artiste_02,Electro,3,1
2,Artiste_03,Pop,3,1
3,Artiste_04,Jazz,3,1


In [18]:
# Vue : recettes par jour de concert
conn.execute("DROP VIEW IF EXISTS v_recettes_par_jour;")
conn.execute("""
CREATE VIEW v_recettes_par_jour AS
SELECT
    date(c.date_heure_debut) AS jour,
    COUNT(b.id_billet) AS nb_billets,
    ROUND(COALESCE(SUM(b.prix), 0), 2) AS recettes
FROM Billet b
JOIN Concert c ON c.id_concert = b.ref_concert
GROUP BY date(c.date_heure_debut)
ORDER BY jour;
""")

run_query(conn, "SELECT * FROM v_recettes_par_jour;")



,jour,nb_billets,recettes
0,2025-07-18,15,945.0
1,2025-07-19,15,945.0
2,2025-07-20,15,945.0
3,2025-07-21,15,945.0


In [ ]:
# Fermeture
conn.close()
print("Connexion fermée")


Connexion fermée
